In [1]:
import time
start_time = time.time()
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider()

In [2]:
# Standard imports
import numpy as np
import pandas as pd
import time
import warnings

# Qiskit 
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes, EfficientSU2
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.circuit.library import QNNCircuit
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, ADAM, SLSQP, AQGD
from qiskit_algorithms.utils import algorithm_globals

# Visualization libraries
from matplotlib import pyplot as plt
import seaborn as sns
from qiskit.visualization import plot_histogram
from IPython.display import clear_output

# Machine learning and preprocessing
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.exceptions import ConvergenceWarning
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

from functools import partial

In [3]:
from sklearn.datasets import load_iris
iris_data = load_iris()
features = iris_data.data
labels = iris_data.target
features = MinMaxScaler().fit_transform(features)
algorithm_globals.random_seed = 4701

train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, train_size = 0.8, random_state = algorithm_globals.random_seed)
num_features = features.shape[1]

In [5]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

adhoc_feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2, entanglement="linear")

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

adhoc_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=adhoc_feature_map)

In [7]:
data_descriptions = ["Full Features", "Missing Sepal Length", "Missing Sepal Width", "Missing Petal Length",
                     "Missing Petal Width"]

In [9]:
full_features = iris_data.data

In [6]:
from qiskit_machine_learning.algorithms import QSVC

qsvc = QSVC(quantum_kernel=adhoc_kernel)

qsvc.fit(train_features, train_labels)

qsvc_score = qsvc.score(test_features, test_labels)

print(f"QSVC classification test score: {qsvc_score}")

QSVC classification test score: 0.8666666666666667


In [10]:
# Assuming full_features and labels are defined earlier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from qiskit_algorithms.utils import algorithm_globals
import time

# Initialize QSVC with adhoc quantum kernel
qsvc_scores = {}
training_times = {}

for i, description in enumerate(data_descriptions):
    # Adjust features based on the description
    if i == 0:
        adjusted_features = full_features.copy()
    else:
        adjusted_features = np.delete(full_features, i-1, axis=1)
    
    # Update the number of features based on the adjusted features
    num_features = adjusted_features.shape[1]
    
    # Scale features and split data
    adjusted_features = MinMaxScaler().fit_transform(adjusted_features)
    train_features, test_features, train_labels, test_labels = train_test_split(
        adjusted_features, labels, train_size=0.8, random_state=algorithm_globals.random_seed)
    
    # Update feature map for QSVC according to the current number of features
    adhoc_feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2, entanglement="linear")
    adhoc_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=adhoc_feature_map)
    qsvc = QSVC(quantum_kernel=adhoc_kernel)
    
    # Measure training time and fit the model
    start_time = time.time()
    qsvc.fit(train_features, train_labels)
    elapsed_time = time.time() - start_time
    training_times[description] = elapsed_time
    
    # Evaluate model performance
    qsvc_score = qsvc.score(test_features, test_labels)
    qsvc_scores[description] = qsvc_score
    
    print(f"{description} - QSVC classification test score: {qsvc_score}, Training time: {elapsed_time:.2f} seconds")

# Plotting or further analysis can be done based on qsvc_scores and training_times


Full Features - QSVC classification test score: 0.8666666666666667, Training time: 50.44 seconds
Missing Sepal Length - QSVC classification test score: 0.9, Training time: 38.94 seconds
Missing Sepal Width - QSVC classification test score: 0.8666666666666667, Training time: 34.46 seconds
Missing Petal Length - QSVC classification test score: 0.9, Training time: 34.90 seconds
Missing Petal Width - QSVC classification test score: 0.8666666666666667, Training time: 33.74 seconds
